In [12]:
import os
os.chdir("../")
%pwd

'/home/ken/Work/xray-lung-classifier'

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_model_lr_image_size: list
    params_model_f_image_size: list
    params_learning_rate: float
    params_classes: int

In [14]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


In [20]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])
        
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_model_lr_image_size= self.params.MODEL_LR_IMAGE_SIZE,
            params_model_f_image_size= self.params.MODEL_F_IMAGE_SIZE,
            params_learning_rate= self.params.LEARNING_RATE,
            params_classes= self.params.CLASSES
        )
        
        return prepare_base_model_config

In [21]:
import os 
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten

In [28]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig, type):
        self.config = config
        self.params_image_size = config.params_model_lr_image_size if type == 'lr' else config.params_model_f_image_size
        
    def get_base_model(self):
        self.model = tf.keras.Sequential()
        self.model.add(Conv2D(8, (5,5), strides=1, padding='same' ,activation='relu', input_shape=self.params_image_size))
        self.model.add(MaxPool2D((2,2), strides=2, padding='same'))

        self.model.add(Conv2D(8, (5,5), strides=1, padding='same' ,activation='relu'))
        self.model.add(MaxPool2D((2,2), strides=2, padding='same'))

        self.model.add(Conv2D(16, (5,5), strides=1, padding='same' ,activation='relu'))
        self.model.add(MaxPool2D((2,2), strides=2, padding='same'))

        self.model.add(Conv2D(16, (5,5), strides=1, padding='same' ,activation='relu'))
        self.model.add(MaxPool2D((2,2), strides=2, padding='same'))

        self.model.add(Flatten())
        self.model.add(Dense(units=self.config.params_classes,  activation='softmax'))
        
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(), 
            metrics=["accuracy"])
        
        self.save_model(path=self.config.base_model_path, model=self.model)
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



In [29]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    model_left = PrepareBaseModel(config=prepare_base_model_config, type='lr')
    model_right = PrepareBaseModel(config=prepare_base_model_config, type='lr')
    model_full = PrepareBaseModel(config=prepare_base_model_config, type='f')
    
    model_left.get_base_model()
    model_right.get_base_model()
    model_full.get_base_model()
    

except Exception as e:
    raise e

[2024-02-26 23:47:54,963: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-26 23:47:54,966: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-26 23:47:54,967: INFO: common: created directory at: artifacts]
[2024-02-26 23:47:54,968: INFO: common: created directory at: artifacts/prepare_base_model]


/home/ken/.virtualenvs/tensorflow/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
